# Structure Byteclass Collection Examples

In [1]:
from byteclasses.types.collections import FixedSizeCollection, structure
from byteclasses.types.primitives.integers import UInt8,UInt16, UChar
from byteclasses.types.primitives.floats import Float32

In [4]:
@structure(byte_order="!")
class SimpleStruct():
    a:UInt8 = UInt8()
    b:UInt16 = UInt16()
    c:UChar = UChar()
    d:Float32 = Float32(1.1)
    e:UInt8 = UInt8()
s: FixedSizeCollection = SimpleStruct()
print(len(s))
print(s)
print(s.data)
b = SimpleStruct(a=UInt8(1), b=UInt16(2), c=UInt8(3), d=Float32(4), e=UInt8(5))
print(b)
print(b.data)

13
SimpleStruct(a=UInt8(0), b=UInt16(0), c=UInt8(0), d=Float32(1.100000023841858), e=UInt8(0))
bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
SimpleStruct(a=UInt8(1), b=UInt16(2), c=UInt8(3), d=Float32(4.0), e=UInt8(5))
bytearray(b'\x01\x00\x02\x00\x03\x00\x00\x00\x00\x00\x80@\x05')


In [4]:
print(dir(s))
print(s.data, f"Length: {len(s)}")
s.a = 1
print(s.a.data)
print(s.data)
s.b = 2
print(s.b.data)
print(s.data)
s.c = 3
print(s.c.data)
print(s.data)
s.d = 4
print(s.d.data)
print(s.data)
s[1] = 6
s[2] = 7
s[3] = 8

print(s.data)
s.data = b"\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00"
print(s.a)
print(s.b)
print(s.a.endianness)

['__annotations__', '__bytes__', '__class__', '__collection_members__', '__collection_params__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_attach_members', '_collection_init', '_data', '_length', '_member_offsets', 'a', 'attach', 'b', 'c', 'd', 'data', 'e']
bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00') Length: 13
b'\x01'
bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
b'\x02\x00'
bytearray(b'\x01\x00\x02\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
b'\x03'
bytearray(b'\x01\x00\x02\x00\x03\x00\x00\x00\xcd\xcc\x8c?\x00')
b'\x00\x00\x80@'
bytearray(b'\x01\x00\x02\x00\x03\x00\x00\x00\x0

In [5]:
print(s)

SimpleStruct(a=UInt8(0), b=UInt16(0), c=UInt8(0), d=Float32(1.100000023841858), e=UInt8(0))
